# Лабораторная работа №6 по предмету "Теория кодирования".
Выполнили студентки:
Журавская Елизавета группа 6404-010302D и Теплякова Юлия группа 6403-010302D.



In [65]:
import numpy as np
from itertools import product
from itertools import combinations
import math
import random
from operator import itemgetter

# 6.1 Написать функции кодирования и декодирования для циклического кода (7,4) с порождающим многочленом 𝑔(𝑥)=1+𝑥2+𝑥3, исправляющего однократные ошибки и провести исследование этого кода для одно-, двух- и трёхкратных ошибок.

In [66]:
def coding_with_errors(generator_poly, word_length, error_count):
    message = np.random.randint(0, 2, word_length, dtype=int)
    print("Исходное сообщение:", message)

    encoded = np.polymul(message, generator_poly) % 2
    error_positions = set()

    while len(error_positions) < error_count:
        error_positions.add(random.randint(0, len(encoded) - 1))

    for pos in error_positions:
        encoded[pos] ^= 1

    return encoded

In [67]:
def is_this_err(error_vector, max_error_size):
    trimmed_error = np.trim_zeros(error_vector, trim='fb')
    return 0 < len(trimmed_error) <= max_error_size

In [68]:
def decoding(generator_poly, max_error_size, received_word, is_packet_error):
    word_length = len(received_word)
    syndrome = np.polydiv(received_word, generator_poly)[1] % 2

    for i in range(word_length):
        error_poly = np.zeros(word_length, dtype=int)
        error_poly[word_length - i - 1] = 1

        shifted_syndrome = np.polymul(syndrome, error_poly) % 2
        residual_syndrome = np.polydiv(shifted_syndrome, generator_poly)[1] % 2

        if is_packet_error and is_this_err(residual_syndrome, max_error_size):
            corrected_word = correct_errors(received_word, residual_syndrome, generator_poly, word_length, i)
            return corrected_word

        if not is_packet_error and sum(residual_syndrome) <= max_error_size:
            corrected_word = correct_errors(received_word, residual_syndrome, generator_poly, word_length, i)
            return corrected_word

    return None

In [69]:
def correct_errors(received_word, syndrome, generator_poly, word_length, index):
    error_poly = np.zeros(word_length, dtype=int)
    error_poly[index - 1] = 1

    correction_poly = np.polymul(error_poly, syndrome) % 2
    corrected_word = (np.polyadd(received_word, correction_poly)) % 2

    quotient = np.polydiv(corrected_word, generator_poly)[0] % 2
    padding = [0] * (len(generator_poly) - len(quotient))
    return np.concatenate([padding, quotient])

In [70]:
n = 7
k = 3
t = 1
g1 = np.array([1, 1, 0, 1])
print(f'g1 = {g1}\n')

for i in range(1, 4):
    print("Количество ошибок:", i)
    u = coding_with_errors(g1, 4, i)
    decoded = decoding(g1, t, u, False)
    print(f'Декодированное сообщение: {decoded}\n')

g1 = [1 1 0 1]

Количество ошибок: 1
Исходное сообщение: [0 1 1 1]
Декодированное сообщение: [0. 1. 1. 0.]

Количество ошибок: 2
Исходное сообщение: [0 0 1 1]
Декодированное сообщение: [0. 0. 0. 1.]

Количество ошибок: 3
Исходное сообщение: [0 0 0 1]
Декодированное сообщение: [0. 0. 0. 1.]



# 6.2 Написать функции кодирования и декодирования для циклического кода (15,9) с порождающим многочленом 𝑔(𝑥)=1+𝑥3+𝑥4+𝑥5+𝑥6, исправляющего пакеты ошибок кратности 3 и провести исследование этого кода для пакетов ошибок длины 1, 2, 3 и 4. Обратите внимание, что пакет ошибок длины t не означает, что все разряды в пределах этого пакета изменятся (см. лекции).


In [71]:
def get_pack_error(word_length, error_size):
    error_vector = np.zeros(word_length, dtype=int)
    start_index = random.randint(0, word_length - 1)
    wrap_offset = 0

    for i in range(error_size):
        if start_index + i == word_length:
            start_index = 0
            wrap_offset = i

        error_vector[start_index + i - wrap_offset] = 1 if i % 2 == 0 else random.randint(0, 1)

    print("Ошибка:", error_vector)
    return error_vector

In [72]:
def coding_with_pack_error(generator_poly, word_length, error_size):
    message = np.random.randint(0, 2, word_length, dtype=int)
    print("Исходное сообщение:", message)

    encoded = np.polymul(message, generator_poly) % 2
    return (encoded + get_pack_error(len(encoded), error_size)) % 2

In [73]:
n2 = 15
k2 = 9
t2 = 3
g2 = np.array([1, 1, 1, 1, 0, 0, 1])
print(f'g2 = {g2}\n')

for i in range(1, 5):
  print("Количество ошибок:", i)
  u = coding_with_pack_error(g2, 9, i)
  decoded = decoding(g2, i, u, True)
  print(f'Декодированное сообщение: {decoded}\n')

g2 = [1 1 1 1 0 0 1]

Количество ошибок: 1
Исходное сообщение: [0 1 1 1 0 1 0 0 0]
Ошибка: [0 1 0 0 0 0 0 0 0 0 0 0 0 0]
Декодированное сообщение: [1. 0. 1. 1. 1. 1. 0. 0.]

Количество ошибок: 2
Исходное сообщение: [1 1 1 0 0 1 0 0 0]
Ошибка: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
Декодированное сообщение: [1. 1. 1. 0. 0. 1. 0. 0. 0.]

Количество ошибок: 3
Исходное сообщение: [0 1 1 0 1 0 0 1 1]
Ошибка: [0 0 0 0 0 1 1 1 0 0 0 0 0 0]
Декодированное сообщение: [1. 1. 0. 1. 0. 1. 0. 1.]

Количество ошибок: 4
Исходное сообщение: [0 0 0 1 0 0 1 0 1]
Ошибка: [0 0 1 1 1 1 0 0 0 0 0 0]
Декодированное сообщение: [0. 1. 0. 1. 1. 0. 1.]

